In [ ]:


import torch

LOAD_TRAIN_PATH = r"D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\Data_processing\TRAIN.pth"

LOAD_VAL_PATH = r"D:\Daniel\for_git\LDCT_git\TSCNN\first_stage_code\Data_processing\VAL.pth"

train_data = torch.load(LOAD_TRAIN_PATH)
val_data = torch.load(LOAD_VAL_PATH)

load_train_patches = train_data['patches'] #(N_total, 3, 64, 64) #0~255
load_train_labels = train_data['masks'] #(N_total, 1, 64, 64) #0 or 1

load_val_patches = val_data['patches'] #(N_total, 3, 64, 64) #0~255
load_val_labels = val_data['masks'] #(N_total, 1, 64, 64) #0 or 1